In [1]:
pip install torch torchvision

Note: you may need to restart the kernel to use updated packages.


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, models
from torchvision.datasets import ImageFolder
from sklearn.model_selection import train_test_split
import os
from PIL import Image

In [3]:
class DeepFakeDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.classes = os.listdir(root_dir)
        self.image_paths = []
        for label, class_name in enumerate(self.classes):
            class_dir = os.path.join(root_dir, class_name)
            for img_name in os.listdir(class_dir):
                self.image_paths.append((os.path.join(class_dir, img_name), label))

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path, label = self.image_paths[idx]
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, label

In [4]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

dataset = DeepFakeDataset(root_dir = r'E:\shruti\dataset\DFD_dataset\Frames', transform=transform)
# Split dataset into train (60%), validation (20%), and test (20%)
train_dataset, temp_dataset = train_test_split(dataset, test_size=0.4, random_state=42)
val_dataset, test_dataset = train_test_split(temp_dataset, test_size=0.5, random_state=42)
# Create test data loader
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [5]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam

In [14]:
# Constants
IMAGE_SIZE = (224, 224)  # ResNet50 default input size
BATCH_SIZE = 32
EPOCHS = 25
LEARNING_RATE = 0.0001
NUM_CLASSES = 2  # Real and Fake

In [9]:
# Create train data loader
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=False)
# Create validation data loader
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [15]:
# Load ResNet50 model
model = models.resnet50(pretrained=True)

# Freeze all layers
for param in model.parameters():
    param.requires_grad = False

# Modify the final layer
num_ftrs = model.fc.in_features
model.fc = torch.nn.Sequential(
    torch.nn.Linear(num_ftrs, 512),
    torch.nn.ReLU(),
    torch.nn.Dropout(0.5),
    torch.nn.Linear(512, NUM_CLASSES)
)

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

In [16]:
# Loss and optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [17]:
# Training loop
for epoch in range(EPOCHS):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # Statistics
        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    train_loss = running_loss / len(train_loader)
    train_acc = 100 * correct / total
    
    # Validation
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for images, labels in val_loader:
            images = images.to(device)
            labels = labels.to(device)
            
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    val_loss = val_loss / len(val_loader)
    val_acc = 100 * correct / total
    
    print(f'Epoch [{epoch+1}/{EPOCHS}], '
          f'Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%, '
          f'Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%')

Epoch [1/25], Train Loss: 0.5977, Train Acc: 68.90%, Val Loss: 0.5086, Val Acc: 73.58%
Epoch [2/25], Train Loss: 0.5037, Train Acc: 75.25%, Val Loss: 0.4618, Val Acc: 76.10%
Epoch [3/25], Train Loss: 0.4634, Train Acc: 76.66%, Val Loss: 0.4360, Val Acc: 79.09%
Epoch [4/25], Train Loss: 0.4496, Train Acc: 77.71%, Val Loss: 0.4260, Val Acc: 79.56%
Epoch [5/25], Train Loss: 0.4437, Train Acc: 79.29%, Val Loss: 0.4169, Val Acc: 80.66%
Epoch [6/25], Train Loss: 0.4305, Train Acc: 80.44%, Val Loss: 0.4105, Val Acc: 81.29%
Epoch [7/25], Train Loss: 0.4244, Train Acc: 79.50%, Val Loss: 0.4121, Val Acc: 80.03%
Epoch [8/25], Train Loss: 0.4138, Train Acc: 80.44%, Val Loss: 0.4073, Val Acc: 80.66%
Epoch [9/25], Train Loss: 0.4008, Train Acc: 81.38%, Val Loss: 0.4071, Val Acc: 80.66%
Epoch [10/25], Train Loss: 0.4119, Train Acc: 80.13%, Val Loss: 0.3995, Val Acc: 80.82%
Epoch [11/25], Train Loss: 0.4048, Train Acc: 81.96%, Val Loss: 0.4015, Val Acc: 80.19%
Epoch [12/25], Train Loss: 0.4025, Train 

In [26]:
torch.save(model.state_dict(), 'deepfake_resnet50.pth')